In [1]:
using FFTW, Random, Test

include("MacroUtils.jl")
using .MacroUtils: cleanup

In [2]:
Random.seed!(1234)
sample_vector_input = rand(32);
sample_tuple_input = Tuple(sample_vector_input);

In [3]:
function test_fft(fft)
    @test FFTW.fft(sample_vector_input) ≈ fft(sample_tuple_input)
end

test_fft (generic function with 1 method)

In [4]:
function fft_rec_c(a, p, b, m)
    n = length(p)
    isone(n) && (b[1] = a[p[1]]; return m)
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]
    y1 = Vector{Complex{Float64}}(undef, n2)
    y2 = Vector{Complex{Float64}}(undef, n2)
    m += 1
    m = fft_rec_c(a, p1, y1, m)
    m = fft_rec_c(a, p2, y2, m)
    for k in 1:n2
        t = w^(k - 1) * y2[k]
        b[k]    = y1[k] + t
        b[k+n2] = y1[k] - t
    end
    return m
end

fft_rec_c (generic function with 1 method)

In [5]:
function fft_rec_c(a)
    n = length(a)
    @assert ispow2(n)
    p = [k for k in 1:n]
    b = Vector{Complex{Float64}}(undef, n)
    m = fft_rec_c(a, p, b, 1)
    return b
end

fft_rec_c (generic function with 2 methods)

In [6]:
fft_rec_c((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [7]:
test_fft(fft_rec_c)

Test Passed

In [8]:
mult_r(x_r::T, x_i::T, y_r::T, y_i::T) where {T} = x_r * y_r - x_i * y_i
mult_i(x_r::T, x_i::T, y_r::T, y_i::T) where {T} = x_r * y_i + x_i * y_r

mult_i (generic function with 1 method)

In [9]:
function fft_rec(a, p, b_r, b_i, m)
    n = length(p)
    isone(n) && begin
        b_r[1] = a[p[1]]
        b_i[1] = 0.0
        return m
    end
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]
    y1_r = Vector{Float64}(undef, n2)
    y1_i = Vector{Float64}(undef, n2)
    y2_r = Vector{Float64}(undef, n2)
    y2_i = Vector{Float64}(undef, n2)
    m += 1
    m = fft_rec(a, p1, y1_r, y1_i, m)
    m = fft_rec(a, p2, y2_r, y2_i, m)
    for k in 1:n2
        wk = w^(k - 1)
        t_r = mult_r(real(wk), imag(wk), y2_r[k], y2_i[k])
        t_i = mult_i(real(wk), imag(wk), y2_r[k], y2_i[k])
        b_r[k] = y1_r[k] + t_r
        b_i[k] = y1_i[k] + t_i
        b_r[k+n2] = y1_r[k] - t_r
        b_i[k+n2] = y1_i[k] - t_i
    end
    return m
end

fft_rec (generic function with 1 method)

In [10]:
function fft_rec(a)
    n = length(a)
    @assert ispow2(n)
    p = [k for k in 1:n]
    b_r = Vector{Float64}(undef, n)
    b_i = Vector{Float64}(undef, n)
    m = fft_rec(a, p, b_r, b_i, 1)
    return [complex(b_r[k], b_i[k]) for k in 1:n]
end

fft_rec (generic function with 2 methods)

In [11]:
fft_rec((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [12]:
test_fft(fft_rec)

Test Passed